In [ ]:

from dyntapy.demand import parse_demand, generate_od_xy, DynamicDemand
from dyntapy.network_data import load_pickle
from dyntapy.dta.assignment import Assignment
from dyntapy.settings import dynamic_parameters
from dyntapy.dta.core.time import SimulationTime
import numpy as np
from dyntapy.settings import default_dynamic_city as city
from dyntapy.visualization import show_demand, show_dynamic_network



In [ ]:
# specifying the time discretization in hours, here 0.25
step_size = dynamic_parameters.network_loading.step_size
# loading from data folder, assumes centroids were added and labelling is correct
g = load_pickle(city + '_grid_centroids')

# generates random demand as a set of lines with a flow variables
geo_jsons = [generate_od_xy(1, city, seed=seed, max_flow=500) for seed in [1, 2]]
times = np.arange(2) # loading times for the different demand patterns
# we now map from specific locations to the existing centroids in the network to generate
# dynamic origin-destination tables.
trip_graphs = [parse_demand(geo_json, g, time) for geo_json, time in zip(geo_jsons, times)]
# these can be visualized on a Map
for trip_graph, time in zip(trip_graphs, times):
    show_demand(trip_graph, title=f'demand at {time}', notebook=True)
# Alternatively, demand can also be added by specifying locations and matrices, see demand module.


AttributeError: '_NetworkLoading' object has no attribute 'step_size'

In [ ]:
# convert everything to internal representations and parse
dynamic_demand = DynamicDemand(trip_graphs, times)
# generating assignment object
assignment = Assignment(g, dynamic_demand, SimulationTime(np.float32(0.0), np.float32(2.0), step_size=step_size))
flows, costs = assignment.run(method='i_ltm_aon')
show_dynamic_network(g, SimulationTime(np.float32(0.0), np.float32(2.0), step_size=step_size),
                link_kwargs={'flows': flows, 'costs': costs}, notebook=True)